In [1]:
from functools import cmp_to_key
import os
import pandas as pd

In [2]:
def compare_file(file1, file2):
    num1 = int(file1.split('.')[0])  
    num2 = int(file2.split('.')[0])  
    if num1 < num2:
        return -1
    elif num1 > num2:
        return 1
    else:
        return 0
def pb_read_appendix1(path="./Problem B/Appendix 1"):
   data=[]
   lower_range=0
   upper_range=2881
   for (_, _, filenames) in os.walk(path):
      temp_save=filenames.copy()
      for idx, ff in enumerate(filenames):
          if ff.startswith("~"):
              temp_save.remove(ff)
      filenames=sorted(temp_save, key=cmp_to_key(compare_file))
      for filename in filenames:
         excel_instance=pd.read_excel(os.path.join(path, filename), sheet_name='Sheet1')
         if len(excel_instance["Var1"].to_list())<(upper_range-lower_range+1):
             continue
         data.append({})
         data[-1]["date"]=[i for i in excel_instance["Var1"].to_list()]
         data[-1]["time"]=[i for i in excel_instance["Var2"].to_list()]
         data[-1]["x"]=[i for i in excel_instance["Var3"].to_list()]
         data[-1]["y"]=[i for i in excel_instance["Var4"].to_list()]
         data[-1]["z"]=[i for i in excel_instance["Var5"].to_list()]
         print(f"{filename} len:{len(data[-1]["date"])}")
   
   for sensor in data:
       sensor["date"]=sensor["date"][lower_range:upper_range]
       sensor["time"]=sensor["time"][lower_range:upper_range]
       sensor["x"]=sensor["x"][lower_range:upper_range]
       sensor["y"]=sensor["y"][lower_range:upper_range]
       sensor["z"]=sensor["z"][lower_range:upper_range]
   return data

In [3]:
appendix1=pb_read_appendix1() #读取所有传感器的数据到一个字典里 只取24:00到4:00的数据！

1.xlsx len:4310
2.xlsx len:4310
3.xlsx len:4310
4.xlsx len:4310
5.xlsx len:4310
6.xlsx len:4310
7.xlsx len:4310
8.xlsx len:4310
9.xlsx len:4310
10.xlsx len:4310
11.xlsx len:4310
12.xlsx len:4310
13.xlsx len:4310
14.xlsx len:4310
15.xlsx len:4310
16.xlsx len:4310
17.xlsx len:4310
18.xlsx len:4310
19.xlsx len:4310
20.xlsx len:4310
21.xlsx len:4310
22.xlsx len:4310
23.xlsx len:4310
24.xlsx len:4310
25.xlsx len:4310
26.xlsx len:4310
27.xlsx len:4310
28.xlsx len:4310
29.xlsx len:4310
30.xlsx len:4310
31.xlsx len:4310
32.xlsx len:4310
33.xlsx len:4310
34.xlsx len:4310
35.xlsx len:4310
36.xlsx len:4310
37.xlsx len:4310
38.xlsx len:4310
39.xlsx len:4310
40.xlsx len:4310
41.xlsx len:4310
42.xlsx len:4310
43.xlsx len:4310
44.xlsx len:4310
45.xlsx len:4310
46.xlsx len:4310
47.xlsx len:4310
48.xlsx len:4310
49.xlsx len:4310
50.xlsx len:4310
51.xlsx len:4310
52.xlsx len:4310
53.xlsx len:4310
54.xlsx len:4310
55.xlsx len:4310
56.xlsx len:4310
57.xlsx len:4310
58.xlsx len:4310
59.xlsx len:4310
60.xls

# 读取格式
读取出来是一个array,每个array是每个传感器的信息
## keys 
1. date$\Rightarrow$一个包含不同点日期的array
2. time$\Rightarrow$一个包含对应时间戳的点
3. x$\Rightarrow$该时间戳下的x值
4. y$\Rightarrow$该时间戳下的y值
5. z$\Rightarrow$该时间戳下的z值

In [4]:
import torch 
import numpy as np
device="cuda"

In [51]:
xs=[]
ys=[]
zs=[]
for sensor in appendix1:
    xs.append([x-sensor["x"][0] for x in sensor["x"]])
    ys.append([y-sensor["y"][0] for y in sensor["y"]])
    zs.append([z-sensor["z"][0] for z in sensor["z"]])
xs=np.array(xs)
ys=np.array(ys)
zs=np.array(zs)
xyz=np.stack((xs,ys,zs),axis=2)
print(xs.shape, xyz.shape)
xyz=np.swapaxes(xyz, 0, 1)
xyz=np.swapaxes(xyz, 1, 2)
print(xyz.shape)

(348, 2881) (348, 2881, 3)
(2881, 3, 348)


In [56]:
num_sensors=len(xs)
from mealpy import GA, IntegerVar


def get_best_sensor_by_ga(idx,data:dict):
    def obj_func(sel_idx):
        sel_idx=round(sel_idx[0])
        sel_x=np.array(data["x"][sel_idx])
        sel_y=np.array(data["y"][sel_idx])
        sel_z=np.array(data["z"][sel_idx])
        sel_x=torch.from_numpy(sel_x).to(device)
        sel_y=torch.from_numpy(sel_y).to(device)
        sel_z=torch.from_numpy(sel_z).to(device)
        org_x=torch.from_numpy(data["x"]).to(device)
        org_y=torch.from_numpy(data["y"]).to(device)
        org_z=torch.from_numpy(data["z"]).to(device)
        mse_x=torch.mean(torch.abs(sel_x-org_x)).cpu().numpy()
        mse_y=torch.mean(torch.abs(sel_y-org_y)).cpu().numpy()
        mse_z=torch.mean(torch.abs(sel_z-org_z)).cpu().numpy()
        return (mse_x+mse_y+mse_z)/3
    problem_dict={
        "obj_func":obj_func,
        "bounds": IntegerVar(lb=[0], ub=[num_sensors-1]),
        "minmax": "min",
    }
    optimizer = GA.BaseGA(epoch=200, pop_size=100, pc=0.85, pm=0.1)
    optimizer.solve(problem_dict)
    print(optimizer.g_best.solution) 
    print(optimizer.g_best.target.fitness) 

In [57]:
get_best_sensor_by_ga(0, {"x":xyz[3][0][:], "y":xyz[3][1][:], "z":xyz[3][2][:]})

2024/11/29 06:41:18 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.
2024/11/29 06:41:19 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 1, Current best: 0.1275402298853587, Global best: 0.1275402298853587, Runtime: 0.17017 seconds
2024/11/29 06:41:19 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 2, Current best: 0.1275402298853587, Global best: 0.1275402298853587, Runtime: 0.14901 seconds
2024/11/29 06:41:19 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 3, Current best: 0.1275402298853587, Global best: 0.1275402298853587, Runtime: 0.12547 seconds
2024/11/29 06:41:19 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 4, Current best: 0.1275402298853587, Global best: 0.1275402298853587, Runtime: 0.14062 seconds
2024/11/29 06:41:19 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 5, Current best: 0.1275402298853587, Global best: 0.127540229

[179.]
0.12705172413805088


In [29]:
def test_idx(data:dict,sel_idx):
    sel_x=data["x"][sel_idx]
    sel_y=data["y"][sel_idx]
    sel_z=data["z"][sel_idx]
    mse_x=np.mean(np.abs((data["x"]-sel_x)))
    mse_y=np.mean(np.abs((data["y"]-sel_y)))
    mse_z=np.mean(np.abs((data["z"]-sel_z)))
    return (mse_x+mse_y+mse_z)/3
test_idx({"x":xyz[1][0][:], "y":xyz[1][1][:], "z":xyz[1][2][:]},43)

0.127103448275824